In [1]:
from sqlite_utils import Database

db_name = "lang_fairy_tale.db"
db = Database(db_name)

# Load in the 
%load_ext sql
%sql sqlite:///$db_name

In [2]:
%%sql
SELECT * FROM books LIMIT 3;

 * sqlite:///lang_fairy_tale.db
Done.


[('The Blue Fairy Book', 'The Bronze Ring', 'Once upon a time in a certain country there lived a king whose palace was surrounded by a spacious garden. But, though the gardeners were many and th ... (20037 characters truncated) ...  pieces as there were nuts upon the mule\'s back.[1]\n\n[1] Traditions Populaires de l\'Asie Mineure. Carnoy et Nicolaides. Paris: Maisonneuve, 1889.', "[1] Traditions Populaires de l'Asie Mineure. Carnoy et Nicolaides. Paris: Maisonneuve, 1889.", 'Once upon a time in a certain country there lived a king whose palace was surrounded by a spacious garden.', "Traditions Populaires de l'Asie Mineure. Carnoy et Nicolaides. Paris: Maisonneuve, 1889.", 0),
 ('The Blue Fairy Book', 'Prince Hyacinth And The Dear Little Princess', 'Once upon a time there lived a king who was deeply in love with a princess, but she could not marry anyone, because she was under an enchantment. So  ... (13405 characters truncated) ... ied the Dear Little Princess, and they lived happily ever after.[1]\n\n[1] Le Prince Desir et la Princesse Mignonne. Par Madame Leprince de Beaumont.', '[1] Le Prince Desir et la Princesse Mignonne. Par Madame Leprince de Beaumont.', 'Once upon a time there lived a king who was deeply in love with a princess, but she could not marry anyone, because she was under an enchantment.', 'Le Prince Desir et la Princesse Mignonne. Par Madame Leprince de Beaumont.', 1),
 ('The Blue Fairy Book', 'East Of The Sun And West Of The Moon', 'Once upon a time there was a poor husbandman who had many children and little to give them in the way either of food or clothing. They were all prett ... (21356 characters truncated) ... ld and silver that they could carry, and moved far away from the castle which lay east of the sun and west of the moon.[1]\n\n[1] Asbjornsen and Moe.', '[1] Asbjornsen and Moe.', 'Once upon a time there was a poor husbandman who had many children and little to give them in the way either of food or clothing.', 'Asbjornsen and Moe.', 2)]

## Entity Extraction...

So what entities can we find in the stories...?!

Let's load in the `spacy` natural language processing toolkit:

In [48]:
#%pip install --upgrade spacy
import spacy
nlp = spacy.load("en_core_web_sm")

Get a dataframe of data frm the database:

In [49]:
q = "SELECT * FROM books"
df = pd.read_sql(q, conn)

df.head()

,book,title,text,last_para,first_line,provenance,chapter_order
0,The Blue Fairy Book,The Bronze Ring,Once upon a time in a certain country there li...,[1] Traditions Populaires de l'Asie Mineure. C...,Once upon a time in a certain country there li...,Traditions Populaires de l'Asie Mineure. Carno...,0
1,The Blue Fairy Book,Prince Hyacinth And The Dear Little Princess,Once upon a time there lived a king who was de...,[1] Le Prince Desir et la Princesse Mignonne. ...,Once upon a time there lived a king who was de...,Le Prince Desir et la Princesse Mignonne. Par ...,1
2,The Blue Fairy Book,East Of The Sun And West Of The Moon,Once upon a time there was a poor husbandman w...,[1] Asbjornsen and Moe.,Once upon a time there was a poor husbandman w...,Asbjornsen and Moe.,2
3,The Blue Fairy Book,The Yellow Dwarf,Once upon a time there lived a queen who had b...,[1] Madame d'Aulnoy.,Once upon a time there lived a queen who had b...,Madame d'Aulnoy.,3
4,The Blue Fairy Book,Little Red Riding Hood,Once upon a time there lived in a certain vill...,"And, saying these words, this wicked wolf fell...",Once upon a time there lived in a certain vill...,,4


Now let's have a go at extracting some entities (this may take some time!):

In [50]:
# Extract a set of entities, rather than a list...
get_entities = lambda desc: {f"{entity.label_} :: {entity.text}" for entity in nlp(desc).ents}

# The full run takes some time....
df['entities'] = df["text"].apply(get_entities)

df.head(10)

,book,title,text,last_para,first_line,provenance,chapter_order,entities
0,The Blue Fairy Book,The Bronze Ring,Once upon a time in a certain country there li...,[1] Traditions Populaires de l'Asie Mineure. C...,Once upon a time in a certain country there li...,Traditions Populaires de l'Asie Mineure. Carno...,0,"{DATE :: Good-day, DATE :: forty days, DATE ::..."
1,The Blue Fairy Book,Prince Hyacinth And The Dear Little Princess,Once upon a time there lived a king who was de...,[1] Le Prince Desir et la Princesse Mignonne. ...,Once upon a time there lived a king who was de...,Le Prince Desir et la Princesse Mignonne. Par ...,1,"{LAW :: the Dear Little Princess up, ORG :: ca..."
2,The Blue Fairy Book,East Of The Sun And West Of The Moon,Once upon a time there was a poor husbandman w...,[1] Asbjornsen and Moe.,Once upon a time there was a poor husbandman w...,Asbjornsen and Moe.,2,"{TIME :: evening, CARDINAL :: thou, TIME :: th..."
3,The Blue Fairy Book,The Yellow Dwarf,Once upon a time there lived a queen who had b...,[1] Madame d'Aulnoy.,Once upon a time there lived a queen who had b...,Madame d'Aulnoy.,3,"{WORK_OF_ART :: Love, TIME :: this minute, CAR..."
4,The Blue Fairy Book,Little Red Riding Hood,Once upon a time there lived in a certain vill...,"And, saying these words, this wicked wolf fell...",Once upon a time there lived in a certain vill...,,4,"{PERSON :: Wolf, ORG :: Grandmamma, DATE :: On..."
5,The Blue Fairy Book,The Sleeping Beauty In The Wood,"There were formerly a king and a queen, who we...","No one dared to tell him, when the Ogress, all...","There were formerly a king and a queen, who we...",,5,"{CARDINAL :: seven, TIME :: four hours, DATE :..."
6,The Blue Fairy Book,"Cinderella, Or The Little Glass Slipper","Once there was a gentleman who married, for hi...",[1] Charles Perrault.,"Once there was a gentleman who married, for hi...",Charles Perrault.,6,"{DATE :: three-quarters, CARDINAL :: six, CARD..."
7,The Blue Fairy Book,Aladdin And The Wonderful Lamp,"There once lived a poor tailor, who had a son ...",[1] Arabian Nights.,"There once lived a poor tailor, who had a son ...",Arabian Nights.,7,"{CARDINAL :: ten thousand, DATE :: forty days,..."
8,The Blue Fairy Book,The Tale Of A Youth Who Set Out To Learn What ...,"A father had two sons, of whom the eldest was ...",[1] Grimm.,"A father had two sons, of whom the eldest was ...",Grimm.,8,"{CARDINAL :: six, TIME :: evening, TIME :: thi..."
9,The Blue Fairy Book,Rumpelstiltzkin,There was once upon a time a poor miller who h...,[1] Grimm.,There was once upon a time a poor miller who h...,Grimm.,9,"{TIME :: the morning, DATE :: The next day, TI..."


*We should probably just do this once and add an appropriate table of entities to the database...*

We can explode these out into a long format dataframe:

In [51]:
from pandas import Series

# Explode the entities one per row...
df_long = df.explode('entities')
df_long.rename(columns={"entities":"entity"}, inplace=True)

# And then separate out entity type and value
df_long[["entity_typ", "entity_value"]] = df_long["entity"].str.split(" :: ").apply(Series)
df_long.head()

,book,title,text,last_para,first_line,provenance,chapter_order,entity,entity_typ,entity_value
0,The Blue Fairy Book,The Bronze Ring,Once upon a time in a certain country there li...,[1] Traditions Populaires de l'Asie Mineure. C...,Once upon a time in a certain country there li...,Traditions Populaires de l'Asie Mineure. Carno...,0,DATE :: Good-day,DATE,Good-day
0,The Blue Fairy Book,The Bronze Ring,Once upon a time in a certain country there li...,[1] Traditions Populaires de l'Asie Mineure. C...,Once upon a time in a certain country there li...,Traditions Populaires de l'Asie Mineure. Carno...,0,DATE :: forty days,DATE,forty days
0,The Blue Fairy Book,The Bronze Ring,Once upon a time in a certain country there li...,[1] Traditions Populaires de l'Asie Mineure. C...,Once upon a time in a certain country there li...,Traditions Populaires de l'Asie Mineure. Carno...,0,DATE :: 1889,DATE,1889
0,The Blue Fairy Book,The Bronze Ring,Once upon a time in a certain country there li...,[1] Traditions Populaires de l'Asie Mineure. C...,Once upon a time in a certain country there li...,Traditions Populaires de l'Asie Mineure. Carno...,0,DATE :: this very day,DATE,this very day
0,The Blue Fairy Book,The Bronze Ring,Once upon a time in a certain country there li...,[1] Traditions Populaires de l'Asie Mineure. C...,Once upon a time in a certain country there li...,Traditions Populaires de l'Asie Mineure. Carno...,0,DATE :: next day,DATE,next day


And explore...

In [52]:
df_long["entity_typ"].value_counts()

DATE           3108
CARDINAL       2536
TIME           1928
ORG            1564
PERSON         1499
ORDINAL         879
GPE             564
WORK_OF_ART     358
NORP            343
QUANTITY        209
LOC             165
PRODUCT         144
FAC             126
MONEY            41
LAW              34
EVENT            28
LANGUAGE         12
Name: entity_typ, dtype: int64

What sort of money has been identified in the stories?

In [53]:
df_long[df_long["entity_typ"]=="MONEY"]["entity_value"].value_counts().head(10)

a penny                     6
a hundred dollars           5
three hundred dollars       4
a few pence                 4
two hundred dollars         3
ten dollars                 2
the hundred dollars         2
fifty dollars               1
only a few hundred yards    1
the hundred marks           1
Name: entity_value, dtype: int64

Dollars? Really??? What about gold coins?! Do I need to train a new classifier?! Or was the original text really like that... Or has the text been got at? *(Maybe I should do my own digitisation project to extract the text from copies of the original books on the Internet Archive? Hmmm.. that could be interesting for when we go on strike...)*

What about other quantities?

In [54]:
df_long[df_long["entity_typ"]=="QUANTITY"]["entity_value"].value_counts().head(10)

one foot           15
a mile             12
a few miles         7
twenty miles        6
three miles         5
a hundred miles     5
two barrels         3
two miles           3
several miles       3
seven miles         3
Name: entity_value, dtype: int64

What people have been identified?

In [55]:
df_long[df_long["entity_typ"]=="PERSON"]["entity_value"].value_counts().head(10)

Fairy       31
Majesty     29
Prince      29
Queen       27
bush        12
wolf        11
Madam       11
Campbell    11
Lang        11
Jack        10
Name: entity_value, dtype: int64

How about geo-political entities (GPEs)?

In [56]:
df_long[df_long["entity_typ"]=="GPE"]["entity_value"].value_counts().head(10)

thou                11
Paris               10
France               9
Japan                8
Greece               7
Denmark              6
Ireland              6
Contes Armeniens     5
Gedichte             5
Finland              5
Name: entity_value, dtype: int64

When did things happen?

In [57]:
df_long[df_long["entity_typ"]=="DATE"]["entity_value"].value_counts().head(10)

one day         188
One day         154
the day          86
three days       79
next day         62
the next day     61
a few days       50
all day          44
many years       41
that day         40
Name: entity_value, dtype: int64

And how about time considerations?

In [58]:
df_long[df_long["entity_typ"]=="TIME"]["entity_value"].value_counts().head(10)

night            117
evening          115
a few minutes     77
morning           69
next morning      63
one morning       58
Next morning      53
the night         47
midnight          44
one night         40
Name: entity_value, dtype: int64

How were things organised?

In [59]:
df_long[df_long["entity_typ"]=="ORG"]["entity_value"].value_counts().head(10)

King        102
Princess     71
Court        48
Prince       42
Grimm        28
King's       23
Quick        18
eagle        12
Majesty      11
I.           10
Name: entity_value, dtype: int64

What's a `NORP`? (Ah... *Nationalities Or Religious or Political groups.)*

In [60]:
df_long[df_long["entity_typ"]=="NORP"]["entity_value"].value_counts().head(10)

German        24
Danish        18
French        14
Russian       11
Indian        10
Christian      9
Italian        8
Portuguese     7
Spanish        7
Chinese        6
Name: entity_value, dtype: int64